In [147]:
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import matplotlib.pylab as plt
plt.style.use('ggplot')
#from feature_engineering import add_retning
from sklearn.model_selection import train_test_split
import pandas as pd
import sys

sys.path.append('../data')

from display_data import import_data

import optuna


In [148]:
def fix_missing_value(df):
    #layout, set everythin to 1
    df['layout'].fillna(int(1.0),inplace=True)



    #found correleation between number of stories and balconies,loggias
    df.loc[(df['stories'] < 7.0)&df['loggias'].isna(),'loggias'] = 0
    df.loc[(df['stories'] < 7.0)&df['balconies'].isna(),'balconies'] = 1

    df.loc[((df['stories'] > 12) & (df['stories'] < 16))|((df['stories'] > 17) & (df['stories'] < 24))&(df['loggias'].isna()),'loggias'] = 1
    df.loc[((df['stories'] > 12) & (df['stories'] < 16))|((df['stories'] > 17) & (df['stories'] < 24))&(df['loggias'].isna()),'balconies'] = 0


    #use the correleation between balconies and loggias to replace nans,
    df.loc[(df['loggias'] == 0.0 )& df['balconies'].isna(),'balconies'] = 1
    df.loc[(df['balconies'] == 0.0 )& df['loggias'].isna(),'loggias'] = 1
    df.loc[(df['loggias'] > 0.0 )& df['balconies'].isna(),'balconies'] = 0
    df.loc[(df['balconies'] > 0.0 )& df['loggias'].isna(),'loggias'] = 0

    #use correlation between material and ceiling groups
    ceiling_under_2 = len(df[((df['material'] == 4)|(df['material'] == 3)) & df['ceiling'].isna()])
    sample = df[df['ceiling'] < 2.6]['ceiling'].sample(n=ceiling_under_2,replace=True).to_numpy()
    df.loc[((df['material'] == 4)|(df['material'] == 3)) & (df['ceiling'].isna()),'ceiling'] = sample
    #use corelation between condition and ceiling groups
    ceiling_under_2 = len(df[(df['condition'] == 1) & df['ceiling'].isna()])
    sample = df[df['ceiling'] < 2.6]['ceiling'].sample(n=ceiling_under_2,replace=True).to_numpy()
    df.loc[(df['condition'] == 1) & df['ceiling'].isna(),'ceiling'] = sample


    #use correleation betwwen ceiling and area
    ceiling_over_3 = len(df[(df['area_total'] > 104)&(df['ceiling'].isna())])
    sample = df[df['ceiling'] > 3.0]['ceiling'].sample(n=ceiling_over_3,replace=True).to_numpy()
    df.loc[(df['area_total'] > 104)&(df['ceiling'].isna() ),'ceiling'] = sample

    ceiling_over_3 = len(df[(df['area_total'] > 75)&(df['ceiling'].isna())])
    sample = df[df['ceiling'] > 2.95]['ceiling'].sample(n=ceiling_over_3,replace=True).to_numpy()
    df.loc[(df['area_total'] > 75)&(df['ceiling'].isna() ),'ceiling'] = sample

    ceiling_under_2 = len(df[(df['area_total'] < 55)&(df['ceiling'].isna())])
    sample = df[df['ceiling'] < 2.8]['ceiling'].sample(n=ceiling_under_2,replace=True).to_numpy()
    df.loc[(df['area_total'] < 55)&(df['ceiling'].isna() ),'ceiling'] = sample

    #seller
    #use correleation between seller and new
    df.loc[(df['new'] == 1 )& df['seller'].isna(),'seller'] = 3

    #use correleation between seller and constructed
    df.loc[(df['constructed'] > 2018)& df['seller'].isna(),'seller'] = 3
    df.loc[(df['constructed'] < 1990)& df['seller'].isna(),'seller'] = 1

    #use correleation between seller and stories
    df.loc[(df['stories'] > 17)& df['seller'].isna(),'seller'] = 3
    df.loc[(df['stories'] < 9)& df['seller'].isna(),'seller'] = 1

    #use correleation between seller and district
    df.loc[((df['district'] == 3)|(df['district']==6))& df['seller'].isna(),'seller'] = 1
    df.loc[((df['district'] == 11)|(df['district']==2))& df['seller'].isna(),'seller'] = 3

    #conditon
    #use correleation between condition and new
    df.loc[(df['new'] == 1)& df['condition'].isna(),'condition'] = 0

    #use correleation between condition and constructed year
    df.loc[(df['constructed'] >= 2017)& df['condition'].isna(),'condition'] = 0
    df.loc[(df['constructed'] <= 1989)& df['condition'].isna(),'condition'] = 1

    #material
    #use correlation between material and district
    df.loc[(df['district'] == 3)& df['material'].isna(),'material'] = 3

    #use correleation between material and constructed year
    df.loc[((df['constructed'] > 1970)& (df['constructed'] <= 1989))&df['material'].isna(),'material'] = 3
    df.loc[(df['constructed'] <= 1970)&df['material'].isna(),'material'] = 0

    #use correleation between stories and material
    df.loc[(df['stories'] < 7)& df['material'].isna(),'material'] = 0
    df.loc[((df['stories'] > 7)&(df['stories'] < 9))& df['material'].isna(),'material'] = 3

    #use correleation between conditon and material
    df.loc[(df['condition'] == 3)& df['material'].isna(),'material'] = 3
    df.loc[(df['condition'] == 0)& df['material'].isna(),'material'] = 2

    #use correleation between parking and material
    df.loc[((df['parking'] == 3)|(df['parking'] == 2))& df['material'].isna(),'material'] = 2

    #user correlation between area and material
    df.loc[(df['area_total'] > 104)& df['material'].isna(),'material'] = 2

    #parking
    #use coreletion between parking and material
    df.loc[((df['material'] == 0)|(df['material'] == 3))& df['parking'].isna(),'parking'] = 1

    #use correleation between stories and parking
    df.loc[((df['stories'] > 30))& df['parking'].isna(),'parking'] = 0
    df.loc[((df['stories'] > 7)&(df['stories'] < 9))& df['parking'].isna(),'parking'] = 1

    #use correleation between radius and parking
    df.loc[(df['radius'] < 0.0547)& df['parking'].isna(),'parking'] = 0
    df.loc[(df['radius'] > 0.251)& df['parking'].isna(),'parking'] = 1

    #windows_court and windows_street
    #windows_street and balconies
    df.loc[(df['balconies'] == 2)& df['windows_street'].isna(),'windows_street'] = 1

    #windows street and area
    df.loc[(df['area_total'] > 106)& df['windows_street'].isna(),'windows_street'] = 1
    df.loc[(df['area_total'] < 47)& df['windows_street'].isna(),'windows_street'] = 0

    #windwos_street and stories
    df.loc[(df['stories'] > 40)& df['windows_street'].isna(),'windows_street'] = 1
    df.loc[(df['stories'] < 9)& df['windows_street'].isna(),'windows_street'] = 0

    #windows_street and windows_court
    df.loc[(df['windows_street'] == 0)& df['windows_court'].isna(),'windows_court'] = 1

    df.loc[((df['windows_street'] == 1)& (df['area_total'] > 75)) & df['windows_court'].isna(),'windows_court'] = 1
    df.loc[((df['windows_street'] == 1)& (df['area_total'] < 47)) & df['windows_court'].isna(),'windows_court'] = 0

    #balconies
    #balconies and stories
    df.loc[((df['stories'] >= 12)& (df['area_total'] <= 16)) & df['balconies'].isna(),'balconies'] = 0
    df.loc[(df['stories'] < 7) & df['balconies'].isna(),'balconies'] = 1
    df.loc[((df['stories'] > 17)& (df['area_total'] <= 24)) & df['balconies'].isna(),'balconies'] = 0

    #matrial and balconies
    df.loc[(df['material'] == 0) & df['balconies'].isna(),'balconies'] = 1
    df.loc[(df['material'] == 2) & df['balconies'].isna(),'balconies'] = 0

    #balconies and constructed
    df.loc[(df['constructed'] < 1970) & df['balconies'].isna(),'balconies'] = 1
    df.loc[(df['constructed'] > 2018) & df['balconies'].isna(),'balconies'] = 0

    #loggias
    #loggias and balconies
    df.loc[(df['balconies'] == 0) & df['loggias'].isna(),'loggias'] = 1
    df.loc[(df['balconies'] > 0) & df['loggias'].isna(),'loggias'] = 0

    #loggias and stories
    df.loc[(df['stories'] < 7) & df['loggias'].isna(),'loggias'] = 0
    df.loc[((df['stories'] >= 17)& (df['area_total'] <= 24)) & df['loggias'].isna(),'loggias'] = 1

    return df

def add_radius(df):
    #adds radius column to dataframe
    df['radius'] = np.sqrt((df['latitude']-55.75)**2 + (df['longitude']-37.55)**2)
    return df

def group_by_feature_and_price(df, feature, data_train = pd.DataFrame()):
    #makes a new column with the mean price for each group to all rows
    if data_train.empty:
        grouped = df.groupby([feature])

        mean_price = grouped['price'].mean()

        df_merged = pd.merge(df,mean_price, on=feature, how='left')

        column_name = str(feature + '_price')

        df[column_name] = df_merged['price_y']
    else:
        grouped = data_train.groupby([feature])

        mean_price = grouped['price'].mean()

        df_merged = pd.merge(df,mean_price, on=feature, how='left')

        column_name = str(feature + '_price')

        df[column_name] = df_merged['price']
    return df

def one_hot_encode(df, column_to_encode):
    """one hots encode for one singel column"""
    encoded_df = pd.get_dummies(df[[column_to_encode]].astype(str))
    df = pd.concat([df,encoded_df],axis=1)
    return df

def one_hot_encode_multiple(df, list_of_columns):
    """takes in multiple columns and runs one hot encode for each column"""
    for column_to_encode in list_of_columns:
        #print(column_to_encode)
        df = one_hot_encode(df, column_to_encode)
    return df

def add_direction(df):
    """adds direction to dataframe, can be one of eight categories (N,S,W,E)"""
    #straight up (north)
    normal_vector = np.array([0,1])
    #normal_vector = np.tile(normal_vector,(df.shape[0],1))
    #normal_vector = normal_vector.reshape((2,-df.shape[0]))
    temp = pd.DataFrame()
    temp['latitude'] = df['latitude']-55.75
    temp['longitude'] = df['longitude']-37.56


    apartment_vector = temp[['latitude','longitude']].to_numpy()


    #print(np.shape(apartment_vector), np.shape(normal_vector))
    angles = []
    for vector in apartment_vector:
        if vector[0] < 0:
            temp_angle = -angle_between(vector,normal_vector)
        else:
            temp_angle = angle_between(vector,normal_vector)
        angles.append(temp_angle)

    angles = [element * 10 for element in angles]


    angles_series = pd.Series(np.array(angles))
    #angles_series.plot.hist()

    df['angle'] = angles_series
    max = df.angle.max()
    min = df.angle.min()
    bins = [min,min*7/8,min*5/8,min*3/8,min/8,max/8,max*3/8,max*5/8,max*7/8,max]
    rounded_bins = [element for element in bins]
    #print(rounded_bins)
    direction = pd.cut(df.angle, bins= rounded_bins,labels=['S','SW','W','NW','N','NE','E','SE','S'],ordered=False)
    df['direction'] = direction
    return df

def unit_vector(vector):
    """returns the unit vector if the vector"""
    return vector/np.linalg.norm(vector)

def angle_between(v1,v2):
    """returns angle between two vectors in radian"""
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u,v2_u),-1,1))

def fix_geo_data(data_test):
    """should just be called on test data"""
    #fix all radius issues
    data_test._set_value(23,'longitude',37.473761)
    data_test._set_value(23,'latitude',55.560891)
    data_test._set_value(90,'longitude',37.473761)
    data_test._set_value(90,'latitude',55.560891)

    data_test._set_value(2511,'longitude',37.478055)
    data_test._set_value(2511,'latitude',55.544046)
    data_test._set_value(5090,'longitude',37.478055)
    data_test._set_value(5090,'latitude',55.544046)
    data_test._set_value(6959,'longitude',37.478055)
    data_test._set_value(6959,'latitude',55.544046)
    data_test._set_value(8596,'longitude',37.478055)
    data_test._set_value(8596,'latitude',55.544046)

    data_test._set_value(4719,'longitude',37.385493)
    data_test._set_value(4719,'latitude',55.853117)

    data_test._set_value(9547,'longitude',37.384711)
    data_test._set_value(9547,'latitude',55.853511)

    data_test._set_value(2529,'longitude',37.464994)
    data_test._set_value(2529,'latitude',55.627666)

    data_test = add_radius(data_test)

    return data_test

def drop_n_largest(data_train):
    """drops 4 largest values, should only be called on training set"""
    #drop 4 largest from training data, maybe not samrt, but we will see
    data_train.drop([3217,21414,15840,13938])
    return data_train

def clean_data(all_data):
    """cleans the data with all the knowledge we have so far"""


    #fix ceiling issues
    all_data.loc[all_data['ceiling'] > 50,'ceiling']*=0.01
    all_data.loc[all_data['ceiling'] > 25, 'ceiling']*=0.1
    all_data.loc[all_data['ceiling'] < 0.5,'ceiling'] = float('NaN')

    return all_data

def add_high_up(df):
    """add exponetital function to determine how high up a building is"""
    high_up = df.floor/df.stories
    high_up_exp = np.exp(high_up) - 1
    euler = np.exp(1)

    df['high_up'] = high_up_exp
    df['high_up'].where(df['high_up'] > euler, euler)


    return df

def log_radius(df):
    df['radius'] = np.log(df['radius'])
    return df

def cluster_geo_data(df,df_test):
    from sklearn.cluster import KMeans
    k_means = KMeans(n_clusters = 100, max_iter = 1000, init='k-means++')

    lat_long_pairs = df[['latitude','longitude']]
    lat_long_pairs_test = df_test[['latitude','longitude']]
    target_data = np.log(df.price)/np.log(15)

    k_means.fit(lat_long_pairs,sample_weight = target_data)
    df['cluster_number'] = k_means.predict(lat_long_pairs)
    df_test['cluster_number'] = k_means.predict(lat_long_pairs_test)

    return df, df_test

In [149]:
data, data_test = import_data()

Y = data.price
test_id = data_test.id

radius = True
# Add radius
if(radius):
    data = add_radius(data)
    data_test = add_radius(data_test)
    #coordinates = ['latitude', 'longitude']
    #data = data.drop(columns = coordinates)
    #data_test = data_test.drop(columns = coordinates)

fix_missing = False
if fix_missing:
    data = fix_missing_value(data)
    data_test = fix_missing_value(data_test)

cleaning = True
if(cleaning):
    data_test = fix_geo_data(data_test)
    data = clean_data(data)
    data_test = clean_data(data_test)


categorical_data = ['seller','layout', 'windows_court', 'windows_street', 'condition', 'building_id','new','district','street',
                    'address', 'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','garbage_chute', 'heating','cluster_number','layout']

data_columns = list(data.columns)
numerical_data = list(set(data_columns)-set(categorical_data))
print(numerical_data)
for column in numerical_data:
    mean = data[column].mean()
    data[column] = data[column].replace(np.NaN, mean)
    if column != 'price':
        mean_test = data_test[column].mean()
        data_test[column] = data_test[column].replace(np.NAN,mean)

#Features
radius = True
district_mean_price = False
ohe = True
direction = True
high_up = True
#add high up
log_r = False
cluster = True
irrelevant_features = True

if(high_up):
    data = add_high_up(data)
    data_test = add_high_up(data_test)





data = data.drop(columns = ['area_living', 'area_kitchen'])
data_test = data_test.drop(columns = ['area_living', 'area_kitchen'])

if(cluster):
    data,data_test = cluster_geo_data(data,data_test)

if(district_mean_price):
    data = group_by_feature_and_price(data,'district')
    data_test = group_by_feature_and_price(data_test,'district',data_train=data)
    data = data.drop(columns=['price', 'id'])

if(direction):
    data = add_direction(data)
    data_test = add_direction(data_test)
    data = one_hot_encode(data, 'direction')
    data_test = one_hot_encode(data_test, 'direction')
    data = data.drop(columns = 'direction')
    data_test = data_test.drop(columns = 'direction')


#One hot encode data
if(ohe):
    encode_categorical_data = ['seller','layout', 'windows_court', 'windows_street', 'condition','new','district',
                     'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','garbage_chute', 'heating','cluster_number']
    data = one_hot_encode_multiple(data,encode_categorical_data)
    data_test = one_hot_encode_multiple(data_test,encode_categorical_data)

if(log_r):
    data = log_radius(data)
    data_test = log_radius(data_test)

if (irrelevant_features):
    data = data.drop(columns = ['rooms', 'bathrooms_shared', 'bathrooms_private', 'balconies', 'loggias', 'phones', 'direction_E', 'direction_NE', 'direction_NW', 'direction_S', 'direction_SE', 'direction_SW', 'direction_W', 'direction_nan', 'seller_0.0', 'seller_1.0', 'seller_2.0', 'seller_3.0', 'layout_0.0', 'layout_1.0', 'layout_2.0', 'layout_nan', 'windows_court_0.0', 'windows_court_1.0', 'windows_court_nan', 'windows_street_0.0', 'windows_street_1.0', 'windows_street_nan', 'condition_0.0', 'condition_1.0', 'condition_nan', 'new_0.0', 'new_1.0', 'new_nan', 'district_1.0', 'district_2.0', 'district_4.0', 'district_5.0', 'district_6.0', 'district_8.0', 'district_9.0', 'district_nan', 'material_0.0', 'material_1.0', 'material_2.0', 'material_3.0', 'material_4.0', 'material_5.0', 'material_6.0', 'material_nan', 'elevator_without_0.0', 'elevator_without_1.0', 'elevator_without_nan', 'elevator_passenger_0.0', 'elevator_passenger_1.0', 'elevator_passenger_nan', 'elevator_service_0.0', 'elevator_service_1.0', 'elevator_service_nan', 'parking_2.0', 'parking_nan', 'garbage_chute_0.0', 'garbage_chute_1.0', 'garbage_chute_nan', 'heating_0.0', 'heating_1.0', 'heating_2.0', 'heating_3.0', 'heating_nan', 'cluster_number_0', 'cluster_number_1', 'cluster_number_10', 'cluster_number_11', 'cluster_number_12', 'cluster_number_13', 'cluster_number_14', 'cluster_number_15', 'cluster_number_16', 'cluster_number_17', 'cluster_number_18', 'cluster_number_19', 'cluster_number_2', 'cluster_number_20', 'cluster_number_21', 'cluster_number_22', 'cluster_number_23', 'cluster_number_24', 'cluster_number_25', 'cluster_number_26', 'cluster_number_27', 'cluster_number_28', 'cluster_number_29', 'cluster_number_3', 'cluster_number_30', 'cluster_number_31', 'cluster_number_32', 'cluster_number_33', 'cluster_number_34', 'cluster_number_35', 'cluster_number_36', 'cluster_number_37', 'cluster_number_38', 'cluster_number_39', 'cluster_number_4', 'cluster_number_40', 'cluster_number_41', 'cluster_number_42', 'cluster_number_43', 'cluster_number_44', 'cluster_number_45', 'cluster_number_46', 'cluster_number_47', 'cluster_number_48', 'cluster_number_49', 'cluster_number_5', 'cluster_number_50', 'cluster_number_51', 'cluster_number_52', 'cluster_number_53', 'cluster_number_55', 'cluster_number_56', 'cluster_number_57', 'cluster_number_58', 'cluster_number_59', 'cluster_number_6', 'cluster_number_60', 'cluster_number_61', 'cluster_number_62', 'cluster_number_63', 'cluster_number_64', 'cluster_number_65', 'cluster_number_66', 'cluster_number_67', 'cluster_number_68', 'cluster_number_69', 'cluster_number_7', 'cluster_number_70', 'cluster_number_71', 'cluster_number_72', 'cluster_number_73', 'cluster_number_74', 'cluster_number_75', 'cluster_number_76', 'cluster_number_77', 'cluster_number_78', 'cluster_number_79', 'cluster_number_8', 'cluster_number_80', 'cluster_number_81', 'cluster_number_82', 'cluster_number_83', 'cluster_number_84', 'cluster_number_85', 'cluster_number_86', 'cluster_number_87', 'cluster_number_88', 'cluster_number_89', 'cluster_number_9', 'cluster_number_90', 'cluster_number_91', 'cluster_number_92', 'cluster_number_93', 'cluster_number_94', 'cluster_number_95', 'cluster_number_96', 'cluster_number_97', 'cluster_number_98', 'cluster_number_99'])
    data_test = data_test.drop(columns = ['rooms', 'bathrooms_shared', 'bathrooms_private', 'balconies', 'loggias', 'phones', 'direction_E', 'direction_NE', 'direction_NW', 'direction_S', 'direction_SE', 'direction_SW', 'direction_W', 'direction_nan', 'seller_0.0', 'seller_1.0', 'seller_2.0', 'seller_3.0', 'layout_0.0', 'layout_1.0', 'layout_2.0', 'layout_nan', 'windows_court_0.0', 'windows_court_1.0', 'windows_court_nan', 'windows_street_0.0', 'windows_street_1.0', 'windows_street_nan', 'condition_0.0', 'condition_1.0', 'condition_nan', 'new_0.0', 'new_1.0', 'new_nan', 'district_1.0', 'district_2.0', 'district_4.0', 'district_5.0', 'district_6.0', 'district_8.0', 'district_9.0', 'district_nan', 'material_0.0', 'material_1.0', 'material_2.0', 'material_3.0', 'material_4.0', 'material_5.0', 'material_6.0', 'material_nan', 'elevator_without_0.0', 'elevator_without_1.0', 'elevator_without_nan', 'elevator_passenger_0.0', 'elevator_passenger_1.0', 'elevator_passenger_nan', 'elevator_service_0.0', 'elevator_service_1.0', 'elevator_service_nan', 'parking_2.0', 'parking_nan', 'garbage_chute_0.0', 'garbage_chute_1.0', 'garbage_chute_nan', 'heating_0.0', 'heating_1.0', 'heating_2.0', 'heating_3.0', 'heating_nan', 'cluster_number_0', 'cluster_number_1', 'cluster_number_10', 'cluster_number_11', 'cluster_number_12', 'cluster_number_13', 'cluster_number_14', 'cluster_number_15', 'cluster_number_16', 'cluster_number_17', 'cluster_number_18', 'cluster_number_19', 'cluster_number_2', 'cluster_number_20', 'cluster_number_21', 'cluster_number_22', 'cluster_number_23', 'cluster_number_24', 'cluster_number_25', 'cluster_number_26', 'cluster_number_27', 'cluster_number_28', 'cluster_number_29', 'cluster_number_3', 'cluster_number_30', 'cluster_number_31', 'cluster_number_32', 'cluster_number_33', 'cluster_number_34', 'cluster_number_35', 'cluster_number_36', 'cluster_number_37', 'cluster_number_38', 'cluster_number_39', 'cluster_number_4', 'cluster_number_40', 'cluster_number_41', 'cluster_number_42', 'cluster_number_43', 'cluster_number_44', 'cluster_number_45', 'cluster_number_46', 'cluster_number_47', 'cluster_number_48', 'cluster_number_49', 'cluster_number_5', 'cluster_number_50', 'cluster_number_51', 'cluster_number_52', 'cluster_number_53', 'cluster_number_55', 'cluster_number_56', 'cluster_number_57', 'cluster_number_58', 'cluster_number_59', 'cluster_number_6', 'cluster_number_60', 'cluster_number_61', 'cluster_number_62', 'cluster_number_63', 'cluster_number_64', 'cluster_number_65', 'cluster_number_66', 'cluster_number_67', 'cluster_number_68', 'cluster_number_69', 'cluster_number_7', 'cluster_number_70', 'cluster_number_71', 'cluster_number_72', 'cluster_number_73', 'cluster_number_74', 'cluster_number_75', 'cluster_number_76', 'cluster_number_77', 'cluster_number_78', 'cluster_number_79', 'cluster_number_8', 'cluster_number_80', 'cluster_number_81', 'cluster_number_82', 'cluster_number_83', 'cluster_number_84', 'cluster_number_85', 'cluster_number_86', 'cluster_number_87', 'cluster_number_88', 'cluster_number_89', 'cluster_number_9', 'cluster_number_90', 'cluster_number_91', 'cluster_number_92', 'cluster_number_93', 'cluster_number_94', 'cluster_number_95', 'cluster_number_96', 'cluster_number_97', 'cluster_number_98', 'cluster_number_99'])

#Drop cat_data
data = data.drop(columns = categorical_data)
data = data.drop(columns = ['latitude','longitude','price', 'id'])
data_test = data_test.drop(columns=['id','latitude','longitude'])
data_test = data_test.drop(columns = categorical_data)
print(data.info())


X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=42)
print('done here')


overlaps=0
for index, value in X_train.items():
    for idx, value in X_test.items():
        if index==idx:
            overlaps+=1
            break

print(overlaps)


'''nan_values = X_train.isna().any()

nan_columns = nan_values.any()




columns_with_nan = X_train.columns[nan_columns].tolist()

for column in X_train.columns:

    if X_train[column].isna().any():

        print(column)'''



['radius', 'loggias', 'bathrooms_shared', 'balconies', 'id', 'phones', 'rooms', 'area_living', 'ceiling', 'latitude', 'price', 'constructed', 'area_total', 'floor', 'stories', 'area_kitchen', 'bathrooms_private', 'longitude']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23285 entries, 0 to 23284
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   area_total         23285 non-null  float64
 1   floor              23285 non-null  float64
 2   ceiling            23285 non-null  float64
 3   constructed        23285 non-null  float64
 4   stories            23285 non-null  float64
 5   radius             23285 non-null  float64
 6   high_up            23285 non-null  float64
 7   angle              23285 non-null  float64
 8   direction_N        23285 non-null  uint8  
 9   seller_nan         23285 non-null  uint8  
 10  condition_2.0      23285 non-null  uint8  
 11  condition_3.0      23285 non-null  u

'nan_values = X_train.isna().any()\n\nnan_columns = nan_values.any()\n\n\n\n\ncolumns_with_nan = X_train.columns[nan_columns].tolist()\n\nfor column in X_train.columns:\n\n    if X_train[column].isna().any():\n\n        print(column)'

In [150]:
def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all()
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5


kaggle=False
if(kaggle):
    Gradient_boosting_model = GradientBoostingRegressor(random_state=0)
    Y = np.log(Y)/np.log(15)
    Gradient_boosting_model.fit(data,Y)
    result = Gradient_boosting_model.predict(data_test)

else:
    param = {"loss":["deviance"],
    "learning_rate": [0.001],
    "min_samples_split": [2],
    "min_samples_leaf": [1],
    "max_depth":[3],
    "max_features":["auto"],
    "criterion": ["friedman_mse"],
    "n_estimators":[50]
    }
    
    
    

In [151]:
if kaggle:
    prediction_kaggle = 15**prediction_kaggle
    submission = pd.DataFrame()
    submission['id'] = test_id
    submission['price_prediction'] = prediction_kaggle
    submission.to_csv('submission_XGB_EDA.csv', index= False)

else:
    import matplotlib.pyplot as plt
    '''prediction = 15**prediction
    rmsle = root_mean_squared_log_error(y_test,prediction)
    print("first run", rmsle)'''

    Gradient_boosting_model = GradientBoostingRegressor(min_samples_split = 3, min_samples_leaf= 1, max_depth= 6, n_estimators= 729, learning_rate= 0.15823029511056225) #beste by default lr=0.1, max_depth=8, n_estimators=750 => 0.137
    #'min_samples_split': 3, 'min_samples_leaf': 1, 'max_depth': 6, 'n_estimators': 729, 'learning_rate': 0.15823029511056225
    y_train_new = np.log(y_train)/np.log(15)
    Gradient_boosting_model.fit(X_train, y_train_new)
    prediction = 15**Gradient_boosting_model.predict(X_test)

    print(root_mean_squared_log_error(y_test, prediction))


    features = Gradient_boosting_model.feature_names_in_
    importances = Gradient_boosting_model.feature_importances_
    zero_count=0
    irrelevant=[]
    for i in range(len(importances)):
        if importances[i]<0.001:
            irrelevant.append(features[i])
    print('zero_count: ',zero_count)
    print('irrelevant features: ', irrelevant)

    def objective(trial):
        Gradient_boosting_model = GradientBoostingRegressor(min_samples_split=trial.suggest_int('min_samples_split',2,4), min_samples_leaf=trial.suggest_int('min_samples_leaf', 1,3),max_depth=trial.suggest_int('max_depth',5,8), n_estimators=trial.suggest_int('n_estimators', 400,750), learning_rate=trial.suggest_float('learning_rate',0.05, 0.20)) #beste by default lr=0.1, max_depth=8, n_estimators=750 => 0.137
        #'min_samples_split': 3, 'min_samples_leaf': 1, 'max_depth': 6, 'n_estimators': 729, 'learning_rate': 0.15823029511056225
        y_train_new = np.log(y_train)/np.log(15)
        Gradient_boosting_model.fit(X_train, y_train_new)
        prediction = 15**Gradient_boosting_model.predict(X_test)

        fig = plt.figure()
        ax = fig.add_axes([0,0,1,1])
        features = Gradient_boosting_model.feature_names_in_
        importances = Gradient_boosting_model.feature_importances_
        ax.bar(features,importances)
        plt.show()

        


        return root_mean_squared_log_error(y_test, prediction)

    #study = optuna.create_study(direction = 'minimize')
    #study.optimize(objective, n_trials=1)
    #0.1363254710562304

0.1398445008856409
zero_count:  0
irrelevant features:  ['cluster_number_54']
